In [26]:
using  TensorKit, RandomMatrix, Plots, LinearAlgebra

In [ ]:
function create_initial(m)
    #creates a 2^m sized lattice vector, which is consistent with a lattice of size 2^(m-1)
    state = zeros(2^m)
    for i in 1:2^m
        t = abs(randn())
        if t>0.5
            state[i] = 1
        else
            state[i] = -1
        end
    end
    return state 
end

function add_ancillas(ρ_state,r)
    """
    create the combined state matrix consisting of 5 registers:
    - |ρ>  : a 2^m sized vector of the actual physical state.
    - |E_i>: a 2^r sized register encoding energy of current state
    - |E_k>: a 2^r sized register encoding energy of new proposed state.
    - |q>  : a 1 qubit sized register encoding the acceptance ancilla.
    - |s>  : a 1 qubit sized register encoding the Q/P ancilla, denoting the measurement result.
    """
    empty_state = zeros(ComplexF64,(2^r,2^r,2,2))
    empty_state[1] = 1
    @tensor state[1,2,3,4,5]:= ρ_state[1] * empty_state[2,3,4,5] # m+2r+2 qubits
    return state
end

function prepare_initial_energy_register(state, Φ)
    @tensor new_state[-1, -2, -3, -4, -5] := Φ[-1, -3; a, b] * state[a, -2, b, -4, -5] 
    return new_state
end

function get_W(r, β, t)
    #constructs a 2^rx2^r matrix where every entry at pos (i,j) is the 2x2 matrix as defined in (2).
    #this essentialy creates all such possible matrices for all possible pairs of E differences inside the range 2^r
    gibbs_energy_difference(z_2, z_1) = min(1, ℯ^(-β  * 2pi * (z_1 - z_2) / (2^r * t))) # Why the extra factor of 2??

    W_mat = zeros(ComplexF64, 2^r, 2^r, 2, 2^r, 2^r, 2)
    for i in 1:2^r
        for j in 1:2^r
            W_mat[i, j, :, i, j, :] = [[(1 - gibbs_energy_difference(i, j))^0.5, gibbs_energy_difference(i, j)^0.5]; [gibbs_energy_difference(i, j)^0.5, -(1 - gibbs_energy_difference(i, j))^0.5]]
        end
    end

    return W_mat
end

function get_iQFT(r)
    iQFT_mat = zeros(ComplexF64, 2^r, 2^r)
    for i in 1:2^r
        for j in 1:2^r
            iQFT_mat[i, j] = ℯ^(1im * (i - 1) * (j - 1) * 2π / 2^r) / 2^(r / 2)
        end
    end
    return iQFT_mat
end

function get_U_exp(state, r, m,J)
    #all U has to do is extract alpha. i can find alpha with a function call to the state, this is fine as its an eigenvect so no
    #collapse. then u exp mat can stay the same to construct the QPE matrix.
    #U_mat = exp(1im * H_mat)
    Jm = J
    Je=J
    #extract the state vector for the H
    ρ_state = state[:, 1, 1, 1, 1]
    E = apply_H(ρ_state,Jm,Je) +2^m*J #account for redefinition of E-scale
    U_vec = exp(1im*E) * ones(2^m)
    U_mat = diagm(U_vec)
    U_exp_mat = zeros(ComplexF64, 2^m, 2^r, 2^m, 2^r)
    for i in 1:2^r
        U_exp_mat[:, i, :, i] = U_mat^(i - 1)
    end
    return U_exp_mat
end
function get_Hadamard_gates(r)
    H = 1 / √2 * [[1 1]; [1 -1]]
    H_gate = TensorMap(H, ℂ^2 → ℂ^2)
    H_gates = H_gate
    for i in 1:r-1
        H_gates = H_gates ⊗ H_gate
    end
    H_gates_mat = Matrix(reshape(H_gates[], 2^r, 2^r))
    return H_gates_mat
end
function apply_H(ρ,Jm,Je)
    #extract |ρ> from the padded state
    #ρ = ρ[:, 1, 1, 1, 1]
    
    n = length(ρ)
    N = n/2
    energy = N*Jm + N*Je #rescale the energy
    for i in 1:n
        if i <= N
            energy += -1* Jm * ρ[i]
        else
            energy += -1* Je * ρ[i]
        end
    end
    return energy
end

function get_Φ(state, r, m,J)
    #QPE
    U_exp = get_U_exp(state, r, m,J)
    iQFT = get_iQFT(r)
    hadamard_gates = get_Hadamard_gates(r)
    
    @tensor Φ[-1, -2; -3, -4] := iQFT[-2, a] * U_exp[-1, a, -3, b] * hadamard_gates[b, -4]
    return Φ
end
function measure_acceptance_qubit(state)
    result = 0
    random_number = rand()
    prob = norm(state[:,:,:,1,:])^2
    println(prob)
    if random_number< prob
        state[:,:,:,2,:] .= 0
        state = state/sqrt(prob)
        result = 0
    else
        state[:,:,:,1,:] .= 0
        state = state/sqrt(1-prob)
        result =1
    end
    state = state[:,1,1,1,1]/norm(state)

    return state, result
end

function apply_Q(state,C,Φ,W, m,r)
    @tensor state_to_be_measured[-1,-2,-3,-4,-5] := W[-2,-3,-4;d,e,f]*Φ[-1,d;b,c] * C[b,a] * state[a, c, e, f, -5] #insert new phi after update and keep first register of new phi, old phi for energy register
    #print("updated state : ")
    #println(state_to_be_measured[:, 1, 1, 1, 1])
    #print("E_0 = ")
    #println(state_to_be_measured[1,:,1,1,1])
    #print("E_1 = ")
    #println(state_to_be_measured[1,1,:,1,1])
    measured_state, result = measure_acceptance_qubit(state_to_be_measured)
    Φ_dag = reshape(reshape(Φ,(2^(m+r),2^(m+r)))',(2^m,2^r,2^m,2^r))
    W_dag = reshape(reshape(W,(2^(2r+1),2^(2r+1)))',(2^r,2^r,2,2^r,2^r,2))
    C_dag = C'

    @tensor new_state[-1,-2,-3,-4,-5] := C_dag[-1,f]*Φ_dag[f,-2;d,e]*W_dag[e,-3,-4;a,b,c]*measured_state[d,a,b,c,-5]
    return new_state, result
end


function get_index(state,update)
    ind_state = update * state
    l = length(ind_state)
    for entry in 1:l 
        if entry == 1
            return entry,false 
        end
    end
    return 0,true
end

function normalise(state,index,GS)
    if GS
        return real.(-1* state/state[index])
    end
    return real.(state/state[index])
end

function apply_L(state,C,Φ,W, m,r)
    Φ_dag = reshape(reshape(Φ,(2^(m+r),2^(m+r)))',(2^m,2^r,2^m,2^r))

    @tensor new_state[-1,-2,-3,-4,-5] := Φ_dag[-1,-2;g,h]*W[h,-3,-4;d,e,f]*Φ[g,d;b,c] * C[b,a] * state[a, c, e, f, -5]
    return new_state
end

function measure_all_ancillas(state, r,m)
    state = reshape(state, (2^m,2^r,2^r,2,2))

    # Measure Q and P ancilla qubits
    random_number = rand()
    prob = norm(state[:,:,:,:,1])^2
    if random_number< prob
        state = state[:,:,:,:,1]/norm(state[:,:,:,:,1])
    else
        state = state[:,:,:,:,2]/norm(state[:,:,:,:,2])
    end

    random_number = rand()
    prob = norm(state[:,:,:,1])^2
    if random_number< prob
        state = state[:,:,:,1]/norm(state[:,:,:,1])
    else
        state = state[:,:,:,2]/norm(state[:,:,:,2])
    end
end

measure_all_ancillas (generic function with 1 method)

In [14]:
r = 5
n = 4
m=2
ρ = create_initial(m)

4-element Vector{Float64}:
 -1.0
  1.0
 -1.0
 -1.0

In [105]:


C = Matrix{ComplexF64}(I(2^m))
C[3,3] = -1
@tensor u[-1] := C[-1,b] * ρ[b]
padded = add_ancillas(ρ, r)
padded_updated = add_ancillas(u, r)
Φ_old = get_Φ(padded,r,m,1)
Φ_new = get_Φ(padded_updated,r,m,1)
#print("test contraction: ")
#@tensor post_qpe_state[-1,-2,-3,-4,-5] := Φ_new[-1,-2,a,b] * padded_updated[a,b,-3,-4,-5]
#println(post_qpe_state[:,1,1,1,1])
#no 2pi -> norm larger than one, array not normalised at all
#2pi -> array normalised but 4th register always e^-27
W = get_W(r,1,1)
padded = prepare_initial_energy_register(padded,Φ_old)
attempt_update,result = apply_Q(padded,C,Φ_new,W,m,r)
print("inshallah: ")
println(attempt_update[:,1,1,1,1])
tre = apply_L(attempt_update,C,Φ_new,W,m,r)
fin = measure_all_ancillas(tre,r,m)
print("bismillah: ")
println(fin[:,1,1,1,1])
#can get somewhat correct probabilities after switching z2-z1 with z1-z2 but now again the normalisation sucks?
#with z2-z1 the probability is good if the update is bad?

0.022445162967838946
inshallah: ComplexF64[-0.0017466949499530143 - 0.003131220550771723im, 0.0017466949499530143 + 0.003131220550771723im, -0.0017466949499530143 - 0.003131220550771723im, -0.0017466949499530143 - 0.003131220550771723im]
bismillah: ComplexF64[-0.001746694949953016 - 0.0031312205507717265im, 0.001746694949953016 + 0.0031312205507717265im, 0.001746694949953016 + 0.0031312205507717265im, -0.001746694949953016 - 0.0031312205507717265im]


In [101]:
a = [1.0, 1.0, 1.0, 1.0, -1.0, -1.0, 1.0, 1.0, 1.0, 1.0, -1.0, -1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, -1.0, 1.0, -1.0, 1.0, 1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, 1.0000000000000002, 1.0000000000000002]

b = [1.0, 1.0, 1.0, 1.0, -1.0, 1.0, 1.0, 1.0, 1.0, 1.0, -1.0, -1.0, -1.0, -1.0, -1.0, 1.0, 1.0, -1.0, 1.0, 1.0, 1.0, 1.0, -1.0, 1.0, 1.0, -1.0, -1.0, 1.0, 1.0, 1.0, -1.0, -1.0]

32-element Vector{Float64}:
  1.0
  1.0
  1.0
  1.0
 -1.0
  1.0
  1.0
  1.0
  1.0
  1.0
  ⋮
  1.0
  1.0
 -1.0
 -1.0
  1.0
  1.0
  1.0
 -1.0
 -1.0

In [103]:
E0 = apply_H(a,1,1)
E1 = apply_H(b,1,1)
print("a = ")
println(E0)
print("b = ")
println(E1)

a = 24.0
b = 24.0
